# Telecom Service Assurance AI Model (Transformer NN) for Latency Insights
Author: Fatih E. NAR

## Introduction
In this notebook, we showcase a machine learning model to create latency predictions for telecom networks.

In [ ]:
# Install the required packages
%pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tf2onnx
import onnx 
import torch
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# Check if CUDA (NVIDIA GPU) is available
if tf.config.list_physical_devices('GPU'):
    device = "/GPU:0"
    print("Using CUDA (NVIDIA GPU)")
else:
    device = "/CPU:0"
    print("Using CPU")

# Example of setting a tensor to the device
x = tf.random.uniform([3, 3])
with tf.device(device):
    x = tf.random.uniform([3, 3])
print(x.device)

# Load the generated data
data = pd.read_csv('data/telecom_sevass_data.csv.xz', compression='xz', parse_dates=['timestamp'])

# Inspect the data for problematic values
print("Initial data inspection:")
print(data.head())
print(data.info())

# Replace string representations of empty lists with NaN
for col in ['latency', 'jitter', 'packet_loss', 'throughput', 'cpu_usage', 'memory_usage']:
    data[col] = data[col].replace('[]', np.nan)

# Ensure all relevant columns are numeric and replace non-numeric values with NaN
for col in ['latency', 'jitter', 'packet_loss', 'throughput', 'cpu_usage', 'memory_usage']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Impute missing values in numeric columns instead of dropping rows
numeric_cols = ['latency', 'jitter', 'packet_loss', 'throughput', 'cpu_usage', 'memory_usage']
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Normalize the data
scaler = MinMaxScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length].values)
    return np.array(sequences)

seq_length = 30  # Length of the sequences (e.g., 30 time steps)
sequences = create_sequences(data[numeric_cols], seq_length)

# Ensure sequences are numeric
for i in range(sequences.shape[0]):
    for j in range(sequences.shape[1]):
        for k in range(sequences.shape[2]):
            if isinstance(sequences[i, j, k], str):
                sequences[i, j, k] = np.nan

# Convert to float32
X = sequences[:, :-1, :].astype(np.float32)  # Input sequences
y = sequences[:, -1, :].astype(np.float32)   # Corresponding labels

# Drop any remaining NaN values
nan_mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(y).any(axis=1)
X = X[nan_mask]
y = y[nan_mask]

# Check shapes of the datasets
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'X_train shape: {X[:int(0.8 * len(X))].shape}')
print(f'X_val shape: {X[int(0.8 * len(X)):].shape}')
print(f'y_train shape: {y[:int(0.8 * len(y))].shape}')
print(f'y_val shape: {y[int(0.8 * len(y)):].shape}')

# Ensure there's sufficient data for training
if len(X) < 32:
    raise ValueError('Not enough data to train the model. Increase the dataset size.')

# Split the data into training and validation sets
train_size = int(0.8 * len(X))
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [ ]:
# Define the Transformer model
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, embed_dim, num_heads, ff_dim, num_layers):
    inputs = Input(shape=input_shape)
    x = Dense(embed_dim)(inputs)
    for _ in range(num_layers):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(input_shape[-1])(x)

    model = Model(inputs, outputs)
    return model

input_shape = (seq_length - 1, X.shape[-1])
model = build_transformer_model(input_shape, embed_dim=64, num_heads=4, ff_dim=128, num_layers=2)

model.compile(optimizer=Adam(learning_rate=0.0001), loss="mse")
model.summary()

# Set the mixed precision policy
policy = Policy('mixed_float16')
set_global_policy(policy)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Increase patience
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stopping]
)

# Evaluate the model
loss = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}')

In [ ]:
# Save the model in Keras format
model.save('data/service_assurance_transformer_model.keras')

# Define a sample input for the model to create an input signature
spec = (tf.TensorSpec((None, seq_length - 1, X.shape[-1]), tf.float32, name="input"),)

# Convert the Keras model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)

# Save the ONNX model to a file
onnx_model_path = "data/service_assurance_transformer_model.onnx"
onnx.save(onnx_model, onnx_model_path)

In [ ]:
# Make predictions
y_pred = model.predict(X_val)

# Rescale the predictions and actual values back to the original scale
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_val_rescaled = scaler.inverse_transform(y_val)

In [ ]:
# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_val_rescaled, y_pred_rescaled)
print(f'MAPE: {mape:.2f}%')

In [ ]:
# Plot the predictions and actual values for a specific metric (e.g., latency)
plt.figure(figsize=(15, 5))
plt.plot(y_val_rescaled[:, 0], label='Actual Latency')
plt.plot(y_pred_rescaled[:, 0], label='Predicted Latency')
plt.title('Actual vs Predicted Latency')
plt.xlabel('Time Steps')
plt.ylabel('Latency')
plt.legend()
plt.show()